# Lab 06 · Embeddings and FAISS

*This lab notebook provides guided steps. All commands are intended for local execution.*

## Objectives
- Document chunking routines are introduced.
- The "text-embedding-004" vectors are stored in a FAISS index.
- A search endpoint returns top results with scores.

## What will be learned
- Document preprocessing for embeddings is rehearsed.
- FAISS index persistence is described.
- Vector search endpoints are surfaced.

## Prerequisites & install
The following commands are intended for local execution.

```bash
cd ai-web/backend
. .venv/bin/activate
pip install faiss-cpu google-generativeai numpy
```

## Step-by-step tasks
### Step 1: Chunking utility
A chunking helper is added so documents are segmented.

In [ ]:
from pathlib import Path
vector_path = Path("ai-web/backend/app/vector.py")
vector_path.write_text('''import json
import os
from pathlib import Path
from typing import List, Tuple

import google.generativeai as genai
import numpy as np
import faiss


DATA_DIR = Path(__file__).resolve().parent / "data"
DATA_DIR.mkdir(parents=True, exist_ok=True)
INDEX_FILE = DATA_DIR / "embeddings.index"
META_FILE = DATA_DIR / "metadata.json"


def chunk_text(text: str, size: int = 400) -> List[str]:
  return [text[i:i + size] for i in range(0, len(text), size) if text[i:i + size].strip()]


def embed_chunks(chunks: List[str]) -> np.ndarray:
  api_key = os.environ.get('GEMINI_API_KEY', '')
  if not api_key:
    raise RuntimeError('A backend API key is required for embeddings.')
  genai.configure(api_key=api_key)
  model = genai.EmbeddingModel(model_name='text-embedding-004')
  vectors = model.embed_content([{ "title": f"Chunk {idx}", "content": chunk } for idx, chunk in enumerate(chunks)])
  return np.array([item.values for item in vectors], dtype=np.float32)


def save_index(chunks: List[str], vectors: np.ndarray) -> None:
  index = faiss.IndexFlatIP(vectors.shape[1])
  faiss.normalize_L2(vectors)
  index.add(vectors)
  faiss.write_index(index, str(INDEX_FILE))
  META_FILE.write_text(json.dumps({"chunks": chunks}))


def load_index() -> Tuple[faiss.Index, List[str]]:
  index = faiss.read_index(str(INDEX_FILE))
  chunks = json.loads(META_FILE.read_text())["chunks"]
  return index, chunks


def search(query: str, top_k: int = 3) -> List[Tuple[str, float]]:
  index, chunks = load_index()
  query_vec = embed_chunks([query])
  faiss.normalize_L2(query_vec)
  scores, neighbors = index.search(query_vec, top_k)
  return [(chunks[i], float(scores[0][pos])) for pos, i in enumerate(neighbors[0])]
''')
print("Vector helper was written.")

### Step 2: Index builder cell
An index is created from a small sample document.

In [ ]:
import sys
from pathlib import Path

sys.path.append(str(Path('ai-web/backend')))
from app.vector import chunk_text, embed_chunks, save_index

sample_text = """This course demonstrates AI in web programming.
The backend relies on FastAPI and Gemini proxies.
Vector search provides relevant snippets.
"""
chunks = chunk_text(sample_text)
vectors = embed_chunks(chunks)
save_index(chunks, vectors)
print('Index was generated with', len(chunks), 'chunks.')

### Step 3: Search endpoint
A FastAPI endpoint is published for vector search.

In [ ]:
from pathlib import Path
main_path = Path("ai-web/backend/app/main.py")
text = main_path.read_text()
if "search_endpoint" not in text:
    addition = '''
from typing import Optional
from .vector import search


@app.get("/api/search")
def search_endpoint(q: str, k: Optional[int] = 3):
    results = search(q, int(k))
    return {"results": [{"text": text, "score": score} for text, score in results]}
'''
    main_path.write_text(text.rstrip() + "
" + addition)
    print("Search endpoint was appended.")
else:
    print("Search endpoint already present.")

## Validation / acceptance checks
```bash
# locally
curl 'http://localhost:8000/api/search?q=fastapi&k=2'
```
- A JSON response containing scored chunks is observed.
- React development mode shows the described UI state without console errors.

## Homework / extensions
- Periodic index rebuild strategies are evaluated for large document sets.
- Client-side rendering of search results is explored.